## Generating wind scenarios using PGscen ##

In this notebook we will use PGscen to create output scenarios for the wind generators in the Texas 7k power grid. This can also be done using the command line tool ```pgscen-wind``` installed as part of PGscen.

We start by setting up our scenario parameters and preparing the input datasets.

In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_wind_data, split_actuals_hist_future, split_forecasts_hist_future)

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]
from IPython.display import display


start_date = '2018-03-03'
cur_path = Path("day-ahead_load.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()

scenario_count = 1000
scen_start_time = pd.to_datetime(' '.join([start_date, "06:00:00"]), utc=True)
scen_timesteps = pd.date_range(start=scen_start_time, periods=24, freq='H')

wind_site_actual_df, wind_site_forecast_df, wind_meta_df = load_wind_data()

(wind_site_actual_hists,
     wind_site_actual_futures) = split_actuals_hist_future(
            wind_site_actual_df, scen_timesteps)

(wind_site_forecast_hists,
     wind_site_forecast_futures) = split_forecasts_hist_future(
            wind_site_forecast_df, scen_timesteps)

fig, (hist_ax, future_ax) = plt.subplots(figsize=(17, 6), nrows=1, ncols=2)
title_args = dict(weight='semibold', size=27)
actual_clr, fcst_clr = "#430093", "#D9C800"
plt_asset = wind_site_actual_df.columns[71]

hist_ax.set_title("History", **title_args)
hist_ax.plot(wind_site_actual_hists[plt_asset][-250:], c=actual_clr)
hist_ax.plot(wind_site_forecast_hists['Forecast_time'][-250:],
             wind_site_forecast_hists[plt_asset][-250:],
             c=fcst_clr)
hist_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

future_ax.set_title("Future", **title_args)
future_ax.plot(wind_site_actual_futures[plt_asset][:250], c=actual_clr)
future_ax.plot(wind_site_forecast_futures['Forecast_time'][:250],
               wind_site_forecast_futures[plt_asset][:250], c=fcst_clr)
future_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

lgnd_ptchs = [Patch(color=actual_clr, alpha=0.53, label="Actuals"),
              Patch(color=fcst_clr, alpha=0.53, label="Forecasts")]

_ = fig.legend(handles=lgnd_ptchs, frameon=False, fontsize=17, ncol=2, loc=8,
               bbox_to_anchor=(0.5, -0.04), handletextpad=0.7)

A crucial difference between the wind generator output model and the load demand model is that the former requires a unique regularization penalty for each pair of wind assets corresponding the physical distance between them, as generator sites that are closer together can be expected to have more similar weather patterns.

We thus use a metadata input dataset in our engine and calculate a normalized set of distances to use for our model.

In [ ]:
from pgscen.engine import GeminiEngine

ge = GeminiEngine(wind_site_actual_hists, wind_site_forecast_hists,
                  scen_start_time, wind_meta_df, 'wind')


import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.GeoDataFrame(
    ge.meta_df, geometry=[Point(xy) for xy in zip(ge.meta_df['longitude'],
                                                  ge.meta_df['latitude'])]
    )

tx = gpd.read_file(Path(data_dir, "MetaData", "Texas_State_Boundary-shp.zip"))
ax = tx.plot(facecolor="none", edgecolor='black', linewidth=3, figsize=(18, 18))
gdf.plot(ax=ax, marker='o', linewidth=0, markersize=143, c='green')
_ = plt.title('ERCOT Wind Sites', fontsize=50)

dist = ge.asset_distance()
display(dist.iloc[24:29, 24:29])

ge.fit(dist.values / (10 * dist.values.max()), 5e-2)

Our fitted model parameters should therefore reflect in part the physical topology of the ERCOT power grid.

In [ ]:
import numpy as np
from itertools import combinations as combn
from shapely.geometry import LineString


asset_cov = ge.model.asset_cov
D = np.sqrt(np.diag(asset_cov))
corr_mat = (D ** -1) * asset_cov * (D ** -1)

cnct_dict = {
    (asset1, asset2): {
        'geometry': LineString([gdf.geometry[asset1], gdf.geometry[asset2]]),
        'corr_val': corr_mat.loc[asset1, asset2]
        }
    for asset1, asset2 in combn(asset_cov.index, 2)
    if corr_mat.loc[asset1, asset2] >= 0.1
    }

edges_gdf = gpd.GeoDataFrame(cnct_dict).transpose()
ax = tx.plot(facecolor="none", edgecolor='black', figsize=(18, 18), linewidth=2.3)
gdf.plot(ax=ax, marker='o', linewidth=0, markersize=71, c='green')
edges_gdf.plot(ax=ax, linewidth=(edges_gdf.corr_val - 0.15).clip(0, 1) ** (1/7),
               alpha=0.31)

_ = ax.axis('off')

Having fit the model, we can now produce scenarios.

In [ ]:
ge.create_scenario(scenario_count, wind_site_forecast_futures)

In [ ]:
from ipywidgets import interact, Dropdown
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def plot_wind_scenarios(wind_site):
    ax = tx.plot(facecolor="none", edgecolor='black', figsize=(18, 18), linewidth=2.3)

    gdf.plot(ax=ax, marker='o', linewidth=0, markersize=71, c='green')
    edges_gdf.plot(ax=ax, linewidth=(edges_gdf.corr_val - 0.15).clip(0, 1) ** (1/7),
                   alpha=0.31)

    cntr_x, cntr_y = ge.meta_df.longitude.mean(), ge.meta_df.latitude.mean()
    site_x, site_y = ge.meta_df.loc[wind_site, ['longitude', 'latitude']]
    xdiff, ydiff = site_x - cntr_x, site_y - cntr_y

    cntr_dist = (xdiff ** 2 + ydiff ** 2) ** 0.5
    site_cos, site_sin = xdiff / cntr_dist, ydiff / cntr_dist
    sctr_x, sctr_y = cntr_x + 8.5 * site_cos, cntr_y + 8.5 * site_sin

    sctr_bbox = sctr_x - 3.1, sctr_y - 1.4, 6.1, 2.8
    sctr_ax = inset_axes(ax, width='100%', height='100%',
                         bbox_to_anchor=sctr_bbox, bbox_transform=ax.transData,
                         loc=10, borderpad=0)
    
    ax.plot([site_x + 0 * site_cos, sctr_x - 3.8 * site_cos],
            [site_y + 0 * site_sin, sctr_y - 2 * site_sin],
            linewidth=2, c='red', alpha=0.53)
    
    for i in range(scenario_count):
        sctr_ax.plot(ge.scenarios['wind'].iloc[i][wind_site],
                     c='black', alpha=0.13, lw=0.2)

    plt_fcst = ge.forecasts['wind'][wind_site]
    sctr_ax.plot(plt_fcst, c=fcst_clr, alpha=0.47, lw=4.1)
    sctr_ax.plot(wind_site_actual_futures.loc[plt_fcst.index, wind_site],
                 c=actual_clr, alpha=0.47, lw=4.1)

    quant_df = ge.scenarios['wind'][wind_site].quantile([0.25, 0.75])
    sctr_ax.fill_between(quant_df.columns, quant_df.iloc[0], quant_df.iloc[1],
                         color='red', alpha=0.31)

    lgnd_ptchs = [Patch(color='black', alpha=0.23, label="Scenarios"),
                  Patch(color='red', alpha=0.41, label="Interquartile Range"),
                  Patch(color=fcst_clr, alpha=0.81, label="Forecast"),
                  Patch(color=actual_clr, alpha=0.81, label="Actual")]

    _ = sctr_ax.legend(handles=lgnd_ptchs, frameon=False,
                       fontsize=11, ncol=2, handletextpad=0.5)

    ax.axis('off')

    
w = Dropdown(options=ge.asset_list, description="Scenarios for generator:",
             layout={'align_self': 'center'}, style={'description_width': 'initial'},
             disabled=False)
_ = interact(plot_wind_scenarios, wind_site=w)